In [1]:
!pip install lxml
!pip install beautifulsoup4
!pip install html5lib 

In [102]:
import pandas as pd, numpy as np
import requests
from bs4 import BeautifulSoup

In [20]:
data=[]
for year in [i for i in range(1924,1993,4)]+[i for i in range(1994,2023,4)]:
    url='https://en.wikipedia.org/wiki/List_of_'+str(year)+'_Winter_Olympics_medal_winners'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for section in str(response.content).split('<h2>')[1:-3]:
        soup=BeautifulSoup(section)
        discipline=soup.find('span').text
        if 'Note' not in discipline:
            if 'See also' not in discipline:
                if 'Medal winner changes' not in discipline:
                    if 'Medal leaders' not in discipline:
                        gender=[i.text for i in soup.findAll('h3')]
                        dfs=pd.read_html(str(soup))
                        for i in range(len(dfs)):
                            if gender==[]:
                                dfs[i]['Gender']='Unknown'
                            else:
                                dfs[i]['Gender']=gender[i]
                            dfs[i]['Discipline']=discipline
                        df=pd.concat(dfs)
                        df['Year']=year
                        df['Games']='Winter'
                        data.append(df)
    print(year)

1924
1928
1932
1936
1940
1944
1948
1952
1956
1960
1964
1968
1972
1976
1980
1984
1988
1992
1994
1998
2002
2006
2010
2014
2018
2022


In [29]:
df=pd.concat(data)
df['Gender'].unique()

array(['Unknown', 'Medal leaders[edit]', 'Multiple medalists[edit]',
       "Men\\'s events[edit]", "Women\\'s events[edit]",
       'Mixed events[edit]', 'Team event[edit]', 'Mixed event[edit]',
       'Mixed team event[edit]'], dtype=object)

In [30]:
data=[]
for year in [i for i in range(2014,2023,4)]:
    url='https://en.wikipedia.org/wiki/List_of_'+str(year)+'_Winter_Olympics_medal_winners'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for section in str(response.content).split('<h2>')[1:-3]:
        soup=BeautifulSoup(section)
        discipline=soup.find('span').text
        if 'Note' not in discipline:
            if 'See also' not in discipline:
                if 'Medal winner changes' not in discipline:
                    if 'Medal leaders' not in discipline:
                        gender=[i.text for i in soup.findAll('h3')]
                        dfs=pd.read_html(str(soup))
                        for i in range(len(dfs)):
                            if gender==[]:
                                dfs[i]['Gender']='Unknown'
                            else:
                                dfs[i]['Gender']=gender[i]
                            dfs[i]['Discipline']=discipline
                        df=pd.concat(dfs)
                        df['Year']=year
                        df['Games']='Winter'
                        data.append(df)
    print(year)

2014
2018
2022


In [201]:
df=pd.concat(data)

In [202]:
df.columns=['Event', 'Gold', 'Silver', 'Bronze', 'Gender', 'Discipline',
       'Year', 'Games', 'Discipline2', 'Games2', 'd1', 'd2',
       'd3']
df=df.drop(['d1','d2','d3'],axis=1)
df['Event']=df[['Event','Games2']].bfill(axis=1)['Event']
df['Event']=df[['Event','Discipline2']].bfill(axis=1)['Event']
df['Gender']=df[['Gender','Discipline2']].bfill(axis=1)['Gender']
df=df.drop(['Games2','Discipline2'],axis=1)
df=df.set_index(['Event', 'Gender', 'Discipline', 'Year','Games']).stack().reset_index()
df.columns=['Event', 'Gender', 'Discipline', 'Year','Games','Medal','Athlete']
df['Event']=df['Event'].str.replace('details','').str[:-2].str.strip()
df.dropna(subset=['Event'])
df['Gender']=df['Gender'].replace('Unknown',np.nan)
df['Gender']=df[['Gender','Event']].bfill(axis=1)['Gender']

In [203]:
g=[]
for i in df['Gender']:
    i=str(i)
    if 'Wom' in i:
        g.append('Women')
    elif 'Men' in i:
        g.append('Men')
    elif 'man' in i:
        g.append('Men')
    elif 'Ladies' in i:
        g.append('Women')
    elif 'Mix' in i:
        g.append('Mixed')
    elif 'Pair' in i:
        g.append('Mixed')
    elif 'danc' in i:
        g.append('Mixed')
    elif 'Team trophy' in i:
        g.append('Mixed')
    elif 'Team relay' in i:
        g.append('Mixed')
    else:
        g.append(np.nan)
df['Gender']=g
df=df[~(df['Event'].astype(str)=='')]

In [204]:
df[df['Gender'].astype(str)=='nan']

,Event,Gender,Discipline,Year,Games,Medal,Athlete
177,Doubles,NaN,Luge,2014,Winter,Gold,Tobias Wendl and Tobias Arlt (GER)\n
178,Doubles,NaN,Luge,2014,Winter,Silver,Andreas Linger and Wolfgang Linger (AUT)\n
179,Doubles,NaN,Luge,2014,Winter,Bronze,Andris \xc5\xa0ics and Juris \xc5\xa0ics (LAT)\n
183,large hill/10 km,NaN,Nordic combined,2014,Winter,Gold,J\xc3\xb8rgen Graabak Norway\n
184,large hill/10 km,NaN,Nordic combined,2014,Winter,Silver,Magnus Hovdal Moan Norway\n
185,large hill/10 km,NaN,Nordic combined,2014,Winter,Bronze,Fabian Riessle Germany\n
186,normal hill/10 km,NaN,Nordic combined,2014,Winter,Gold,Eric Frenzel Germany\n
187,normal hill/10 km,NaN,Nordic combined,2014,Winter,Silver,Akito Watabe Japan\n
188,normal hill/10 km,NaN,Nordic combined,2014,Winter,Bronze,Magnus Krog Norway\n
189,Team large hill/4 \xc3\x97 5 km,NaN,Nordic combined,2014,Winter,Gold,Norway (NOR)Magnus Hovdal MoanH\xc3\xa5vard Kl...


In [199]:
df['Gender'].unique()

array(['Men', 'Women', 'Mixed', 'Doubles', 'large hill/10\xa0km',
       'normal hill/10\xa0km', 'Team large hill/4 \\xc3\\x97 5\xa0km',
       'Team event', 'Individual large hill/10\xa0km',
       'Individual normal hill/10\xa0km', 'Team event[edit]',
       'Open doubles', 'Individual large hill / 10\xa0km',
       'Individual normal hill / 10\xa0km',
       'Team large hill / 4 x 5\xa0km'], dtype=object)